In [1]:
import gym
import tensorflow as tf
import numpy as np

from collections import deque
import random

/Users/Damcy/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Hyper Parameters
ENV_NAME = "CartPole-v0"
EPISODE = 10000
STEP = 1000
TEST_TIME = 100

In [3]:
INITIAL_EPSILON = 0.99
FINAL_EPSILON = 0.01
REPLAY_SIZE = 10000
BATCH_SIZE = 32
HIDDEN_UNIT = 20
LEARNING_RATE = 0.0001
GAMMA = 0.9
class DQN:
    def __init__(self, env):
        # init experience replay
        self.replay_buff = deque()
        # parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.hidden_layer_unit = HIDDEN_UNIT
        self.lr = LEARNING_RATE
        self.batch_size = BATCH_SIZE
        self.gamma = GAMMA
        
        self.create_Q_network()
        self.create_training_method()
        
        # init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())
    
    def create_Q_network(self):
        # network
        self.state_input = tf.placeholder("float32", [None, self.state_dim])
        hidden_layer = tf.contrib.layers.fully_connected(
            inputs=self.state_input,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
            biases_initializer=tf.constant_initializer(0.01),
            num_outputs=self.hidden_layer_unit,
            scope='hidden_layer', activation_fn=tf.nn.relu
        )
        self.Q_value = tf.contrib.layers.fully_connected(
            inputs=hidden_layer,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
            biases_initializer=tf.constant_initializer(0.01),
            num_outputs=self.action_dim,
            scope='output_layer', activation_fn=None
        )
    
    def create_training_method(self):
        self.action_input = tf.placeholder("float32", [None, self.action_dim])
        self.y_input = tf.placeholder("float32", [None])
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value, self.action_input), reduction_indices=1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.cost)
    
    def perceive(self, state, action, reward, next_state, done):
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        self.replay_buff.append((state, one_hot_action, reward, next_state, done))
        if len(self.replay_buff) > REPLAY_SIZE:
            self.replay_buff.popleft()
        if len(self.replay_buff) > BATCH_SIZE:
            self.train_Q_network()
    
    def train_Q_network(self):
        self.time_step += 1
        # get mini-batch
        mini_batch = random.sample(self.replay_buff, self.batch_size)
        state_batch = [data[0] for data in mini_batch]
        action_batch = [data[1] for data in mini_batch]
        reward_batch = [data[2] for data in mini_batch]
        next_state_batch = [data[3] for data in mini_batch]
        
        #
        y_batch = []
        next_state_Q_value_batch = self.Q_value.eval(feed_dict={self.state_input: next_state_batch})
        for i in range(0, self.batch_size):
            done = mini_batch[i][4]
            if done:
                y_batch.append(reward_batch[i])
            else:
                y_batch.append(reward_batch[i] + self.gamma * np.max(next_state_Q_value_batch[i]))
        
        # training
        self.optimizer.run(feed_dict={
            self.y_input: y_batch,
            self.action_input: action_batch,
            self.state_input: state_batch
        })
    
    def egreedy_action(self, state):
        Q_value = self.Q_value.eval(feed_dict={
            self.state_input: [state]
        })[0]
        self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / (EPISODE * STEP * 1.0)
        if random.random() <= self.epsilon:
            return random.randint(0, self.action_dim - 1)
        else:
            return np.argmax(Q_value)
    
    def action(self, state):
        return np.argmax(self.Q_value.eval(feed_dict={
            self.state_input: [state]
        })[0])

In [ ]:
def main():
    env = gym.make(ENV_NAME)
    agent = DQN(env)
    
    for i in range(2):
        env.reset()
        for _ in range(100):
            env.render()
            state, reward, done, _ = env.step(env.action_space.sample())

    for episode in range(0, EPISODE):
        # testing
        if episode % 100 == 0:
            total_reward = 0
            for i in range(0, TEST_TIME):
                state = env.reset()
                for step in range(0, STEP):
                    env.render()
                    action = agent.action(state)
                    state, reward, done, _ = env.step(action)
                    total_reward += reward
                    if done:
                        break
            avg_reward = total_reward / TEST_TIME
            print('episode: ', episode, 'Evaluation Average Reward:', avg_reward)
            if avg_reward >= 200:
                break

        state = env.reset()
        # training
        for step in range(0, STEP):
            action = agent.egreedy_action(state)
            next_state, reward, done, _ = env.step(action)
            reward_agent = -1 if done else 0.1
            agent.perceive(state, action, reward, next_state, done)
            state = next_state
            if done:
                break

In [ ]:
main()

You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


episode:  0 Evaluation Average Reward: 9.28
episode:  100 Evaluation Average Reward: 9.36
episode:  200 Evaluation Average Reward: 9.48
episode:  300 Evaluation Average Reward: 9.37
episode:  400 Evaluation Average Reward: 9.35
episode:  500 Evaluation Average Reward: 9.4
